<a href="https://colab.research.google.com/github/tngus4334/LG-Aimers/blob/main/y%ED%94%BC%EC%B2%98_%EB%B3%84_%EC%95%99%EC%83%81%EB%B8%94_%EB%B9%84%EC%9C%A8_%EC%B0%BE%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제출용 코드가 아니라 훈련 데이터셋 스플릿해서 점수 확인하는 코드에 사용하셔야 합니다.

In [ ]:
model_gb = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=0.012688323956682181, max_depth=9,
                          max_features=0.5317748828007712,
                          min_impurity_decrease=1.2229206693892366e-09,
                          min_samples_leaf=2, min_samples_split=4,
                          n_estimators=238, random_state=42,
                          subsample=0.916882961309737)).fit(train_x, train_y)

In [ ]:
model_xgb = MultiOutputRegressor(xgb.XGBRegressor(olsample_bytree=0.8, silent=1, max_depth=7, learning_rate= 0.06, n_estimators=250,
                           random_state =7, nthread = -1, n_jobs=-1)).fit(train_x, train_y)

In [ ]:
model_lgb = MultiOutputRegressor(lgb.LGBMRegressor(objective='regression',n_estimators=250, bagging_fraction=0.7, learning_rate=0.06,
                         max_depth=7, subsample=0.7, feature_fraction=0.9, boosting_type='gbdt',
                         colsample_bytree=0.5, reg_lambda=5, n_jobs=-1)).fit(train_x, train_y)

In [ ]:
model_cat = MultiOutputRegressor(CatBoostRegressor(n_estimators=1300, learning_rate=0.03,
                                   max_depth=7)).fit(train_x, train_y)

In [ ]:
pred_gb = model_gb.predict(test_x)
pred_xgb = model_xgb.predict(test_x)
pred_lgb = model_lgb.predict(test_x)
pred_cat = model_cat.predict(test_x)

In [ ]:
from sklearn.metrics import mean_squared_error as metrics
from sklearn.metrics import mean_squared_error
from math import sqrt

y피처 별 최소 ensemble 찾기

In [ ]:
allscore=[]
for i in range(0,11) :
  for j in range(0,11) :
    for k in range(0,11) :
      for n in range(0,11) :
        if(i+j+k+n==10) :
          ensemble = pred_gb*(i/10) + pred_xgb*(j/10)+ pred_lgb*(k/10)+pred_cat*(n/10)
          sample_submission = pd.read_csv(DATA_PATH +'sample_submission.csv')
          indexlist = test_y.index
          submit = sample_submission.loc[indexlist]
          for idx, col in enumerate(submit.columns):
            if col=='ID':
              continue
            submit[col] = ensemble[:,idx-1]
          #print('Done.')
          
          list_yname = ['Y_01']     #Y_01부터 Y_14까지 수정 후 한 번 씩 실행
          all_nrmse = []
          for l in range(0, 1) :
            rmse = mean_squared_error(test_y[list_yname[l]], submit[list_yname[l]], squared=False)
            nrmse = rmse/np.mean(np.abs(test_y[list_yname[l]]))
            all_nrmse.append(nrmse)
          score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
          print(i)
          print(j)
          print(k)
          print(n)
          print(score)
          allscore.append(score)
        else :
          continue

print(min(allscore))        #출력된 y 피처 별 최소값에 해당하는 앙상블 비율 x:x:x:x 찾아서 따로 기록해야 함       